# Test code to exercise dataset handling with PSF event types.


In [80]:
import os
os.environ['CALDB'] = os.path.expandvars('${BASE_DIR}/irfs/caldb/CALDB')
os.environ['TIMING_DIR'] = os.path.expandvars('${GLAST_EXT}/extFiles/v0r9/jplephem')

In [87]:
from uw.like2 import configuration, bands, dataset
import numpy as np
map(reload,(configuration,bands,dataset))

[<module 'uw.like2.configuration' from '/nfs/farm/g/glast/u/wallacee/pointlike-dev/python/uw/like2/configuration.pyc'>,
 <module 'uw.like2.bands' from '/nfs/farm/g/glast/u/wallacee/pointlike-dev/python/uw/like2/bands.pyc'>,
 <module 'uw.like2.dataset' from '/nfs/farm/g/glast/u/wallacee/pointlike-dev/python/uw/like2/dataset.py'>]

## Specify a working directory and create it if needed

In [82]:
cwd = os.path.expandvars('/tmp/test_psf_binning')
if not os.path.exists(cwd):
    os.mkdir(cwd)
os.chdir(cwd)

## Create configuration files

Write out config.txt files in individual subdirectories. Configurations are set up to test:

  * Loading a pass 7 dataset
  * Loading a pass 8 dataset with front/back binning
  * Loading a pass 8 dataset with PSF binning using the 'dataspec' config key 
  * Loading a pass 8 dataset with PSF binning using the 'datadict' config key 
    
All configurations use a single month of data. Paths to data files 

In [83]:
cfg7 = dict(
    dataspec=dict(
        ft1files=os.path.expandvars('/afs/slac/g/glast/groups/catalog/P7_P202/Source/P7_P202_BASE_Source_001_zmax100.fits'),
        ft2files=os.path.expandvars('/afs/slac/g/glast/groups/catalog/P7_P202/ft2_4years.fits'),
        binfile=os.path.join(cwd,'P202_4bpd_fb.fits'),
        ltcube=os.path.join(cwd,'ltcube.fits'),
        binsperdec=4,
        
    ),
    irf= 'P7REP_SOURCE_V15',
    
    input_model = dict(path='$FERMI/skymodels/P202/uw30'),

    diffuse= dict(
        ring    =  dict(filename='template_4years_P7_v15_repro_v3_4bpd.zip',
                    correction='galactic_correction_v3.csv', 
                    systematic=0.0316), 
        isotrop = '4Y_SOURCE_**_V9_EGBfree.txt', 
        SunMoon = 'SunMoon_4years_zmax100.fits', 
        limb    = 'AziLimb:azi_limb_P7_v0_*.txt',
        ),
    
    extended= 'Extended_archive_v13',
)

cfg8_fb = dict(
    dataspec=dict(
        ft1files=os.path.expandvars('$FERMI/data/P8_P302/zmax105/P302_Source_001_zmax105.fits'),
        ft2files=os.path.expandvars('/afs/slac/g/glast/groups/catalog/P8_P302/ft2_2008.fits'),
        binfile=os.path.join(cwd,'P302_4bpd_fb.fits'),
        ltcube=os.path.join(cwd,'ltcube.fits'),
        binsperdec=4,
    ),
    irf= 'P8R2_SOURCE_V6',
    
    input_model = dict(path='$FERMI/skymodels/P302_7years/uw1002'),

    diffuse = dict(
    ring    = dict(type='HealpixCube', 
            filename='/nfs/slac/g/ki/ki20/elliott/Pass8_GC/gcfit/results/P8_P302_ultraclean_veto_z90/galprop/models/'\
                'diffuse_model_flux_P8_P302_ultraclean_veto_z90_umap_ifilter_Galprop_5rings_IC123_geomLoopI_PS_P8uw963_adaptive_ps_mask_P8uw963_DM_Cusp_n2p5_ISO_plaw_pnorm_psfall.fits',
            correction='galactic_correction_uw1002A.csv', 
            systematic=0.0316), 
    isotrop = dict(type='IsotropicList', filename='isotropic_source_*_4years_P8V3.txt',
            correction='isotropic_correction_*_uw965.csv'),
    limb    = None, 
    SunMoon = 'template_SunMoon_6years_zmax100.fits', 
    ),
    
    extended= 'Extended_archive_v14',
)

cfg8_psf = dict(
    dataspec=dict(
        ft1files=[os.path.expandvars('$FERMI/data/P8_P302/zmax105/P302_Source_001_zmax105.fits')],
        ft2files=[os.path.expandvars('/afs/slac/g/glast/groups/catalog/P8_P302/ft2_2008.fits')],
        binfile=os.path.join(cwd,'P302_4bpd_psf.fits'),
        ltcube=os.path.join(cwd,'ltcube.fits'),
        binsperdec=4,
        psf_event_types=True,
    ),
    irf= 'P8R2_SOURCE_V6',
    
    input_model = dict(path='$FERMI/skymodels/P302_7years/uw1002'),

    diffuse = dict(
    ring    = dict(type='HealpixCube', 
            filename='/nfs/slac/g/ki/ki20/elliott/Pass8_GC/gcfit/results/P8_P302_ultraclean_veto_z90/galprop/models/'\
                'diffuse_model_flux_P8_P302_ultraclean_veto_z90_umap_ifilter_Galprop_5rings_IC123_geomLoopI_PS_P8uw963_adaptive_ps_mask_P8uw963_DM_Cusp_n2p5_ISO_plaw_pnorm_psfall.fits',
            correction='galactic_correction_uw1002A.csv', 
            systematic=0.0316), 
    isotrop = dict(type='IsotropicList', filename='isotropic_source_*_4years_P8V3.txt',
    correction='isotropic_correction_*_uw965.csv'),
    limb    = None, 
    SunMoon = 'template_SunMoon_6years_zmax100.fits', 
    ),
    
    extended= 'Extended_archive_v14',
)
dataspec_dict = dict(P302_psf_test=dict(data_name="Pass8, P302, 1 month, PSF event types",
                 ft1files = '$FERMI/data/P8_P302/zmax105/P302_Source_001_zmax105.fits',
                 ft2files = '/afs/slac/g/glast/groups/catalog/P8_P302/ft2_2008.fits',
                 binsperdec=4,
                 binfile = 'P302_4bpd_psf.fits',
                 ltcube =  'ltcube.fits', 
                 psf_event_types = True,
                 legacy = True,))
cfg8_psf_datadict = dict(
    datadict=dict(
        dataname='P302_psf_test',
        ft1files=[os.path.expandvars('$FERMI/data/P8_P302/zmax105/P302_Source_001_zmax105.fits')],
        ft2files=[os.path.expandvars('/afs/slac/g/glast/groups/catalog/P8_P302/ft2_2008.fits')],
        binfile=os.path.join(cwd,'P302_4bpd_psf.fits'),
        ltcube=os.path.join(cwd,'ltcube.fits'),
        binsperdec=4,
        psf_event_types=True,
    ),
    irf= 'P8R2_SOURCE_V6',
    
    input_model = dict(path='$FERMI/skymodels/P302_7years/uw1002'),

    diffuse = dict(
    ring    = dict(type='HealpixCube', 
            filename='/nfs/slac/g/ki/ki20/elliott/Pass8_GC/gcfit/results/P8_P302_ultraclean_veto_z90/galprop/models/'\
                'diffuse_model_flux_P8_P302_ultraclean_veto_z90_umap_ifilter_Galprop_5rings_IC123_geomLoopI_PS_P8uw963_adaptive_ps_mask_P8uw963_DM_Cusp_n2p5_ISO_plaw_pnorm_psfall.fits',
            correction='galactic_correction_uw1002A.csv', 
            systematic=0.0316), 
    isotrop = dict(type='IsotropicList', filename='isotropic_source_*_4years_P8V3.txt',
    correction='isotropic_correction_*_uw965.csv'),
    limb    = None, 
    SunMoon = 'template_SunMoon_6years_zmax100.fits', 
    ),
    
    extended= 'Extended_archive_v14',
)


for cfg,cfgdir in zip([cfg7,cfg8_fb,cfg8_psf],['P202_4bpd_fb','P302_4bpd_fb','P302_4bpd_psf']):
    if not os.path.exists(cfgdir):
        os.mkdir(cfgdir)
    with open(os.path.join(cfgdir,'config.txt'),'w') as f:
        f.write(str(cfg))
cfg,cfgdir = cfg8_psf_datadict,'P302_4bpd_psf_datadict'
if not os.path.exists(cfgdir):
    os.mkdir(cfgdir)
with open(os.path.join(cfgdir,'config.txt'),'w') as f:
    f.write(str(cfg))
with open(os.path.join(cfgdir,'dataspec.py'),'w') as f:
    f.write(str(dataspec_dict))
    

In [89]:
if not os.path.basename(os.getcwd())=='P202_4bpd_fb':
    os.chdir(os.path.join(cwd,'P202_4bpd_fb'))
config_p7 = configuration.Configuration(quiet=True)

if not os.path.basename(os.getcwd())=='P302_4bpd_fb':
    os.chdir(os.path.join(cwd,'P302_4bpd_fb'))

config_fb = configuration.Configuration(quiet=True)

if not os.path.basename(os.getcwd())=='P302_4bpd_psf':
    os.chdir(os.path.join(cwd,'P302_4bpd_psf'))

config_spec = configuration.Configuration(quiet=True)

if not os.path.basename(os.getcwd())=='P302_4bpd_psf_datadict':
    os.chdir(os.path.join(cwd,'P302_4bpd_psf_datadict'))
config_dict = configuration.Configuration(quiet=True)


processing cuts:  ZENITH_ANGLE zenith_cut
ft1_cut DSTYP3: ZENITH_ANGLE
DSUNI3: deg
DSVAL3: 0:100
DSREF3: None
processing cuts:  THETA theta_cut
ft1_cut None
processing cuts:  EVENT_CLASS event_class_cut
ft1_cut None
Failed dss comparison:
 ltcube DSTYP1: BIT_MASK(EVENT_CLASS,128,P8R2)
DSUNI1: DIMENSIONLESS
DSVAL1: 1:1
DSREF1: None
DSTYP2: TIME
DSUNI2: s
DSVAL2: TABLE
DSREF2: :GTI
DSTYP3: ENERGY
DSUNI3: MeV
DSVAL3: 10:1000000
DSREF3: None
DSTYP4: ZENITH_ANGLE
DSUNI4: deg
DSVAL4: 0:105
DSREF4: None
DSTYP5: THETA
DSUNI5: deg
DSVAL5: :66.4
DSREF5: None,
 FT1 DSTYP1: TIME
DSUNI1: s
DSVAL1: TABLE
DSREF1: :GTI
DSTYP2: ENERGY
DSUNI2: MeV
DSVAL2: 30:1000000
DSREF2: None
DSTYP3: ZENITH_ANGLE
DSUNI3: deg
DSVAL3: 0:100
DSREF3: None
DSTYP4: THETA
DSUNI4: deg
DSVAL4: :66.4
DSREF4: None
DSTYP5: BIT_MASK(EVENT_CLASS,2)
DSUNI5: DIMENSIONLESS
DSVAL5: 1:1
DSREF5: None
on iteration 0
processing cuts:  ZENITH_ANGLE zenith_cut
ft1_cut DSTYP4: ZENITH_ANGLE
DSUNI4: deg
DSVAL4: 0:105
DSREF4: None
processing cu

## Test loading data
### Not working with PSF types yet 

Create a BandSet for each Configuration, and load the data. 


In [91]:
from uw.like2 import from_healpix, bands
roi_index = 840
for config in (config_p7, config_fb, config_spec):
    roi_bands = bands.BandSet(config,roi_index,load=True)

IndexError: list index out of range